# Tool Use

:::{note}
See [here](pkg-info-autogen-agentchat) for installation instructions.
:::

:::{warning}
🚧 Under construction 🚧
:::

In [5]:
from autogen_agentchat.agents import ToolUseAssistantAgent
from autogen_agentchat.teams.group_chat import RoundRobinGroupChat
from autogen_core.components.models import OpenAIChatCompletionClient
from autogen_core.components.tools import FunctionTool

In [6]:
async def get_weather(city: str) -> str:
    return "Sunny"


get_weather_tool = FunctionTool(get_weather, description="Get the weather for a city")

In [7]:
assistant = ToolUseAssistantAgent(
    "Weather_Assistant",
    model_client=OpenAIChatCompletionClient(model="gpt-4o-mini"),
    tool_schema=[get_weather_tool.schema],
)
team = RoundRobinGroupChat([assistant], tools=[get_weather_tool])
result = await team.run("What's the weather in New York?")
print(result)

--------------------------------------------------------------------------------
user:
What's the weather in New York?
--------------------------------------------------------------------------------
Weather_Assistant:
[FunctionCall(id='call_I8mFF4D73eoC3hhO81ldmIG3', arguments='{"city":"New York"}', name='get_weather')]
--------------------------------------------------------------------------------
tool_agent:
[FunctionExecutionResult(content='Sunny', call_id='call_I8mFF4D73eoC3hhO81ldmIG3')]
--------------------------------------------------------------------------------
Weather_Assistant:
The weather in New York is sunny. 

TERMINATE
TeamRunResult(result='The weather in New York is sunny. \n\nTERMINATE')
